In [5]:

from langchain_community.document_loaders import PyPDFLoader

# 문서 로드
loader = PyPDFLoader('../data/KCI_FI003153549_p5.pdf')
documents = loader.load()

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks = text_splitter.split_documents(documents)

In [9]:
len(chunks)

3

In [7]:
from langchain_ollama import OllamaEmbeddings

# * bge-m3 임베딩 모델 준비
# OllamaEmbeddings 랭체인 문서: https://python.langchain.com/docs/integrations/text_embedding/ollama/
embedding_model = OllamaEmbeddings(
    model="bge-m3",
)

In [8]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
import uuid

# 문서 요약
prompt_text = '다음 문서의 요약을 생성하세요:\n\n{doc}'

prompt = ChatPromptTemplate.from_template(prompt_text)

# 도커를 이용하고 있으므로 base_url을 지정해주어야 함
llm = OllamaLLM(
    model="gemma3:latest",
    base_url="http://localhost:11434")
    
summarize_chain = {
    'doc': lambda x: x.page_content} | prompt | llm | StrOutputParser()

summaries = summarize_chain.batch(chunks, {'max_concurrency': 5}) # 최대 5개의 작업을 동시에 실행하도록 제한하는 설정

id_key = 'doc_id'

# 문서와 동일한 길이가 필요하므로 summaries에서 chunks로 변경
doc_ids = [str(uuid.uuid4()) for _ in chunks]

# 각 요약은 doc_id를 통해 원본 문서와 연결
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [10]:
summary_docs[1]

Document(metadata={'doc_id': 'ba50a144-811b-4d7f-9111-fd7f219c511d'}, page_content='이 문서는 의료기기 임상시험에 특화된 Private LLM 접근 방법을 제안합니다. 이 방법은 다음과 같은 네 가지 핵심 단계로 구성됩니다.\n\n1. **도메인 특화 데이터셋 구축:** 의료기기 임상시험 전문가로부터 수집된 158건의 문서(총 11,954 페이지)를 활용하여 규제 문서(30%), 교육 자료(20%) 등으로 구성된 데이터셋을 구축합니다.\n2. **LLM 모델 튜닝:** 구축된 데이터셋을 활용하여 LLM 모델을 튜닝합니다.\n3. **도메인 특화 프롬프트 적용:** 의료기기 임상시험 분야의 특성을 반영하는 프롬프트를 사용합니다.\n4. **도메인 특화 기능 구현:** 의료기기 임상시험 분야에 최적화된 기능들을 구현합니다.\n\n이러한 접근 방식은 분석 시간을 줄이고, 상세한 인사이트를 제공하며, 대규모 데이터셋을 효과적으로 처리할 수 있도록 설계되었습니다. 핵심 목표는 의료기기 임상시험 분야에서 LLM의 최적 성능을 달성하는 것입니다.\n')

In [11]:
from langchain_community.vectorstores import FAISS

# 벡터 저장소(FAISS)에 요약을 인덱싱
vectorstore = FAISS.from_documents(summary_docs, embedding_model)

In [12]:
# FAISS 벡터 저장소를 로컬 파일에 저장
vectorstore.save_local("./faiss_index")

In [13]:
from langchain.storage import LocalFileStore, create_kv_docstore

# 원문은 별도 docstore 폴더로 관리
byte_store = LocalFileStore("./docstore")
store = create_kv_docstore(byte_store)

In [14]:
# 원본 문서를 문서 저장소에 저장 (doc_id로 연결)
# 문서 내용은 유니코드 형식(\u)로 저장
store.mset(list(zip(doc_ids, chunks)))

In [15]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

# MultiVectorRetriever 구성
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [16]:
# 예시 질의
query = "본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수와 문서 유형별 비율은 어떻게 되나요?"
# query = "Advance RAG 기법이 임상시험 데이터 분석에서 수행하는 주요 역할은 무엇인가요?"
# query = "본 연구에서 Private LLM 성능을 평가하기 위해 사용한 지표 3가지는 무엇인가요?"
# query = "국내에서 LLM을 임상시험에 적용한 대표적인 기관과 그 적용 사례를 2가지 이상 말해보세요."
# query = "ROUGE 평가에서 Private LLM과 ChatGPT의 Recall 값은 각각 얼마였나요?"

#### 요약 문서 검색

In [17]:
# 벡터 저장소가 요약을 검색
sub_docs = retriever.vectorstore.similarity_search(query, k=2)

In [18]:
sub_docs

[Document(id='319f62cd-ce1f-4e5e-98ff-06864af5a1aa', metadata={'doc_id': '4892d06d-2a4a-4ac8-9b63-091714c999d7'}, page_content='이 문서는 의료기기 임상시험 분야에 특화된 Private LLM 구축을 위한 데이터셋 구축 과정에 대한 내용입니다. 총 11,954 페이지(111,954 페이지) 규모의 데이터셋은 의료기기 임상시험 전문가로부터 수집되었으며, 다음 내용으로 분류됩니다.\n\n*   **규제 문서 (30%):** FDA, EMA, PMDA 가이드라인 등\n*   **교육 자료 (20%):** 임상시험 수행자 교육 매뉴얼 등\n*   **프로토콜 및 보고서 (25%):** 임상시험 프로토콜, CSR 템플릿 등\n*   **의료기기 특화 문서 (15%):** 의료기기 임상시험 계획서, 기술 문서 등\n*   **기타 (10%):** 윤리위원회 관련 문서, 환자 동의서 템플릿 등\n\n데이터셋은 높은 도메인 적합성, 다양성, 응용 가능성을 가지고 있으며, 규제, 프로토콜 설계, 데이터 관리 등 의료기기 임상시험 전반의 지식을 포괄합니다.'),
 Document(id='939b69fc-41fb-497f-8c0b-4cb3fdf9f7c8', metadata={'doc_id': 'ba50a144-811b-4d7f-9111-fd7f219c511d'}, page_content='이 문서는 의료기기 임상시험에 특화된 Private LLM 접근 방법을 제안합니다. 이 방법은 다음과 같은 네 가지 핵심 단계로 구성됩니다.\n\n1. **도메인 특화 데이터셋 구축:** 의료기기 임상시험 전문가로부터 수집된 158건의 문서(총 11,954 페이지)를 활용하여 규제 문서(30%), 교육 자료(20%) 등으로 구성된 데이터셋을 구축합니다.\n2. **LLM 모델 튜닝:** 구축된 데이터셋을 활용하여 LLM 모델을 튜닝합니다.\n3. **도메인 특화 프롬프트 적용:** 의료기기 임상시

In [19]:
print(sub_docs[0].page_content)

이 문서는 의료기기 임상시험 분야에 특화된 Private LLM 구축을 위한 데이터셋 구축 과정에 대한 내용입니다. 총 11,954 페이지(111,954 페이지) 규모의 데이터셋은 의료기기 임상시험 전문가로부터 수집되었으며, 다음 내용으로 분류됩니다.

*   **규제 문서 (30%):** FDA, EMA, PMDA 가이드라인 등
*   **교육 자료 (20%):** 임상시험 수행자 교육 매뉴얼 등
*   **프로토콜 및 보고서 (25%):** 임상시험 프로토콜, CSR 템플릿 등
*   **의료기기 특화 문서 (15%):** 의료기기 임상시험 계획서, 기술 문서 등
*   **기타 (10%):** 윤리위원회 관련 문서, 환자 동의서 템플릿 등

데이터셋은 높은 도메인 적합성, 다양성, 응용 가능성을 가지고 있으며, 규제, 프로토콜 설계, 데이터 관리 등 의료기기 임상시험 전반의 지식을 포괄합니다.


#### 원본 문서 검색

In [20]:
# 더 큰 원본 문서 청크를 반환
retrieved_docs = retriever.invoke(query)

In [21]:
retrieved_docs

[Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20250909104709', 'source': '../data/KCI_FI003153549_p5.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='의료기기 임상시험 분야의 도메인 특성에 맞게 튜닝하\n기 위해 의료기기 임상시험 전문가로부터 총 158개의 문\n서(총 11,954 페이지)를 수집하였다. 수집된 문서는 다음\n과 같이 분류된다:\n\x9f 규제 문서 (30%): FDA, EMA, PMDA 가이드라인, \nGCP 문서 등\n\x9f 교육 자료 (20%): 임상시험 수행자 교육 매뉴얼, 온라\n인 강의 자료 등\n\x9f 프로토콜 및 보고서 (25%): 임상시험 프로토콜, CSR \n(Clinical Study Report) 템플릿 등\n\x9f 의료기기 특화 문서 (15%): 의료기기 임상시험 계획\n서, 기술문서 등\n\x9f 기타 (10%): 윤리위원회 관련 문서, 환자 동의서 템플\n릿 등\n1.2 Validity of Collected Data\n수집된 데이터셋은 의료기기 임상시험에 특화된 \nPrivate LLM 구축을 위해 도메인 적합성과 다양성, 그리\n고 응용 가능성 측면에서 높은 타당성을 갖추고 있다. 총 \n111,954페이지로 구성된 데이터는 의료기기 임상시험의 \n규제, 프로토콜 설계, 데이터 관리 등 전반적인 지식을 포\n괄하며, 국제 표준과 실제 임상시험 환경에서 발생할 수 \n있는 다양한 시나리오를 반영하도록 설계되었다.\n수집된 데이터는 규제 문서(30%), 교육 자료(20%), 프\n로토콜 및 보고서(25%), 의료기기 특화 문서(15%), 기타'),
 Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:202